In [4]:
# 引入 library
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np

In [5]:
# 讀取資料
path = 'S_All_A1_E1/'
mat = 'S1_A1_E1'
data = []
for i in range(1,28,1):
    file = path + 'S' + str(i) + '_A1_E1'
    data.append(sio.loadmat(file))
    plt.close()
print('資料讀取完畢！')

資料讀取完畢！


In [6]:
# 讀取受試者資料
# subject_emg[受試者編號][emg編號][訊號]
# subject_emg[受試者編號][狀態]
subject_emg = []
subject_restimulus = []
for sub_data in enumerate(data):
    print('第' + str(sub_data[0]+1) + '受試者資料讀取')
    subject_emg.append(sub_data[1]['emg'].T)
    subject_restimulus.append(sub_data[1]['restimulus'])
print('受試者資料讀取完畢！')

第1受試者資料讀取
第2受試者資料讀取
第3受試者資料讀取
第4受試者資料讀取
第5受試者資料讀取
第6受試者資料讀取
第7受試者資料讀取
第8受試者資料讀取
第9受試者資料讀取
第10受試者資料讀取
第11受試者資料讀取
第12受試者資料讀取
第13受試者資料讀取
第14受試者資料讀取
第15受試者資料讀取
第16受試者資料讀取
第17受試者資料讀取
第18受試者資料讀取
第19受試者資料讀取
第20受試者資料讀取
第21受試者資料讀取
第22受試者資料讀取
第23受試者資料讀取
第24受試者資料讀取
第25受試者資料讀取
第26受試者資料讀取
第27受試者資料讀取
受試者資料讀取完畢！


In [97]:
# 設定窗口大小與位移量(單位:sample point)
sliding_window_size = 80
sliding_duration = 40

In [98]:
# 資料預處理
import math
import pywt

x_data = []
y_data = []
count = 0
for emg, restimulus in zip(subject_emg, subject_restimulus):
    count += 1
    print('第' + str(count) + '位受試者資料預處理')
    total_len = len(restimulus)
    # math.ceil -> 無條件進位
    sliding_times = math.ceil((total_len - sliding_window_size) / sliding_duration) + 1

    # 資料分割 + 特徵提取
    window_begin = 0
    for i in range(sliding_times):
    #   特徵提取
        feature_matrix = []
        for e in emg:
            emg_segment = e[window_begin:window_begin+sliding_window_size]
        #   使用多階小波包轉換
        #   小波包基底: db5
        #   層數: 4
            wp = pywt.WaveletPacket(data=emg_segment, wavelet='db5', mode='symmetric', maxlevel=4)
        #   對第四層每一節點做能量值計算
            wavelet_energy = []
            for j in [node.path for node in wp.get_level(wp.maxlevel, 'natural')]:
                wavelet_energy.append(np.sum( (np.array(wp[j].data)) ** 2 ))
            feature_matrix.append(wavelet_energy)
        x_data.append(feature_matrix)
    #   標標籤
        restimulus_segment = restimulus[window_begin:window_begin+sliding_window_size]
    #   np.sqeeze()把矩陣內的單維向量的框框消掉
        counts = np.bincount(np.squeeze(restimulus_segment))
        #返回眾數(注意:此方法只有非負數列才可使用)
        label_action_ID = np.argmax(counts)
        y_data.append(label_action_ID)
        window_begin = window_begin + sliding_duration
print('資料預處理完畢！共', len(x_data), '筆資料')
print('資料標籤數量分布：', np.bincount(np.squeeze(y_data)))

第1位受試者資料預處理
第2位受試者資料預處理
第3位受試者資料預處理
第4位受試者資料預處理
第5位受試者資料預處理
第6位受試者資料預處理
第7位受試者資料預處理
第8位受試者資料預處理
第9位受試者資料預處理
第10位受試者資料預處理
第11位受試者資料預處理
第12位受試者資料預處理
第13位受試者資料預處理
第14位受試者資料預處理
第15位受試者資料預處理
第16位受試者資料預處理
第17位受試者資料預處理
第18位受試者資料預處理
第19位受試者資料預處理
第20位受試者資料預處理
第21位受試者資料預處理
第22位受試者資料預處理
第23位受試者資料預處理
第24位受試者資料預處理
第25位受試者資料預處理
第26位受試者資料預處理
第27位受試者資料預處理
資料預處理完畢！共 68272 筆資料
資料標籤數量分布： [38636  2569  2517  2830  2373  2374  2477  2494  2615  2386  2301  2213
  2487]


In [99]:
# 讓每一種標籤的資料都平均一點
import random
x_filter_data = []
y_filter_data = []
for i in range(len(x_data)):
    if y_data[i] == 0:
        if random.randint(1,15) == 1:
            x_filter_data.append(x_data[i])
            y_filter_data.append(y_data[i])
    else:
        x_filter_data.append(x_data[i])
        y_filter_data.append(y_data[i])
x_data = x_filter_data
y_data = y_filter_data
del x_filter_data
del y_filter_data
print('資料篩選完成')
print('資料數量： x -> ', len(x_data), ', y ->', len(y_data))
print('資料標籤分佈：', np.bincount(np.squeeze(y_data)))

資料篩選完成
資料數量： x ->  32128 , y -> 32128
資料標籤分佈： [2492 2569 2517 2830 2373 2374 2477 2494 2615 2386 2301 2213 2487]


In [100]:
# 正規化
from sklearn import preprocessing
# 34095 x 10 x 16
x_data = list(preprocessing.scale(np.array(x_data).reshape((-1, 16))).reshape(-1, 10, 16))

In [101]:
for i in range(len(x_data)):
    x_data[i] = [x_data[i]]

In [102]:
import torch
import torch.utils.data as Data

# 訓練批次
EPOCH = 100
# 一個Batch的樣本數
BATCH_SIZE = 512
# Learning rate
LR = 0.0001

# 先转换成 torch 能识别的 Dataset
torch_dataset = Data.TensorDataset(torch.FloatTensor(x_data), torch.LongTensor(y_data))

# 把 dataset 放入 DataLoader
loader = Data.DataLoader(
    dataset=torch_dataset,      # torch TensorDataset format
    batch_size=BATCH_SIZE,      # mini batch size
    shuffle=True,               # 要不要打乱数据 (打乱比较好)
)

In [93]:
import torch.nn as nn
import torchvision
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # nn.Sequential()可以快速搭建神經網路
        # 卷積運算所使用的mask一般稱為kernel map，這邊為5x5的map
        # stride決定kernel map一次要走幾格
        # 上面用5x5的kernel map去跑28x28的圖片，卷積完會只剩下26x26，故加兩層
        # zero-padding 讓圖片大小相同
        self.conv1 = nn.Sequential( # input shape(channel=1, height=28, weight=28)
            nn.Conv2d(
                in_channels = 1, # 輸入信號的通道
                out_channels = 4, # 卷積產生的通道
                kernel_size = (3, 5), # 卷積核的尺寸
                stride = 1, # 卷積步長
                padding = (1,2) # 輸入的每一條邊補充0的層數
            ),  # output shape(4, 10, 16)
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            # input shape(4, 10, 16)
            nn.Conv2d(4, 8, (3,5), 1, (1,2)), # output shape(8, 10, 16)
            nn.ReLU()
        )
        self.hidden = nn.Linear(8*10*16, 128)
        self.hidden2 = nn.Linear(128, 128)
        self.hidden3 = nn.Linear(128, 128)
        self.out = nn.Linear(128, 13) # fully connected layer
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1) # 展平多維卷積圖層 (batch_size, 32*10*16)
        x = self.hidden(x)
        x = self.hidden2(x)
        x = self.hidden3(x)
        output = F.softmax(self.out(x))
        return output
    
cnn = CNN()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 4, kernel_size=(3, 5), stride=(1, 1), padding=(1, 2))
    (1): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(4, 8, kernel_size=(3, 5), stride=(1, 1), padding=(1, 2))
    (1): ReLU()
  )
  (hidden): Linear(in_features=1280, out_features=128, bias=True)
  (hidden2): Linear(in_features=128, out_features=128, bias=True)
  (hidden3): Linear(in_features=128, out_features=128, bias=True)
  (out): Linear(in_features=128, out_features=13, bias=True)
)


In [103]:
# optimize all cnn parameters
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
# the target label is not one-hotted
# pytorch 的 CrossEntropyLoss 會自動把張量轉為 one hot形式
loss_func = nn.CrossEntropyLoss()

# training and testing
for epoch in range(EPOCH):
    print('第', epoch, '次訓練')
    # enumerate : 枚舉可列舉對象．ex.
    # A = [a, b, c]
    # list(enumerate(A)) = [(0,a), (1,b), (2,c)]
    for step, (b_x, b_y) in enumerate(loader):
        output = cnn(b_x)
        loss = loss_func(output, b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (step % 50 == 0):
            print('step:' + str(step))
            print('loss:' + str(loss))
print('訓練完成！！')

第 0 次訓練
step:0
loss:tensor(2.3723, grad_fn=<NllLossBackward>)


/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


step:50
loss:tensor(2.3927, grad_fn=<NllLossBackward>)
第 1 次訓練
step:0
loss:tensor(2.3668, grad_fn=<NllLossBackward>)
step:50
loss:tensor(2.3828, grad_fn=<NllLossBackward>)
第 2 次訓練
step:0
loss:tensor(2.3633, grad_fn=<NllLossBackward>)
step:50
loss:tensor(2.3698, grad_fn=<NllLossBackward>)
第 3 次訓練
step:0
loss:tensor(2.3445, grad_fn=<NllLossBackward>)
step:50
loss:tensor(2.3967, grad_fn=<NllLossBackward>)
第 4 次訓練
step:0
loss:tensor(2.3684, grad_fn=<NllLossBackward>)
step:50
loss:tensor(2.3398, grad_fn=<NllLossBackward>)
第 5 次訓練
step:0
loss:tensor(2.3408, grad_fn=<NllLossBackward>)
step:50
loss:tensor(2.3714, grad_fn=<NllLossBackward>)
第 6 次訓練
step:0
loss:tensor(2.3667, grad_fn=<NllLossBackward>)
step:50
loss:tensor(2.3833, grad_fn=<NllLossBackward>)
第 7 次訓練
step:0
loss:tensor(2.3812, grad_fn=<NllLossBackward>)
step:50
loss:tensor(2.3783, grad_fn=<NllLossBackward>)
第 8 次訓練
step:0
loss:tensor(2.3468, grad_fn=<NllLossBackward>)
step:50
loss:tensor(2.3666, grad_fn=<NllLossBackward>)
第 9 次訓練
s

step:50
loss:tensor(2.3770, grad_fn=<NllLossBackward>)
第 71 次訓練
step:0
loss:tensor(2.3301, grad_fn=<NllLossBackward>)
step:50
loss:tensor(2.3638, grad_fn=<NllLossBackward>)
第 72 次訓練
step:0
loss:tensor(2.3401, grad_fn=<NllLossBackward>)
step:50
loss:tensor(2.3271, grad_fn=<NllLossBackward>)
第 73 次訓練
step:0
loss:tensor(2.3521, grad_fn=<NllLossBackward>)
step:50
loss:tensor(2.3620, grad_fn=<NllLossBackward>)
第 74 次訓練
step:0
loss:tensor(2.3353, grad_fn=<NllLossBackward>)
step:50
loss:tensor(2.3017, grad_fn=<NllLossBackward>)
第 75 次訓練
step:0
loss:tensor(2.3094, grad_fn=<NllLossBackward>)
step:50
loss:tensor(2.3112, grad_fn=<NllLossBackward>)
第 76 次訓練
step:0
loss:tensor(2.3392, grad_fn=<NllLossBackward>)
step:50
loss:tensor(2.3298, grad_fn=<NllLossBackward>)
第 77 次訓練
step:0
loss:tensor(2.3480, grad_fn=<NllLossBackward>)
step:50
loss:tensor(2.3502, grad_fn=<NllLossBackward>)
第 78 次訓練
step:0
loss:tensor(2.3164, grad_fn=<NllLossBackward>)
step:50
loss:tensor(2.3534, grad_fn=<NllLossBackward>)
第

In [108]:
from sklearn import metrics
begin = 0
end = len(y_data)
# end = 500
test_output = cnn(torch.FloatTensor(x_data[begin:end]))
pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
y_cut_data = y_data[begin:end]
# 混淆矩陣
confusion_matrix = metrics.confusion_matrix(y_cut_data,pred_y)
print('混淆矩陣')
print('true/predict')
print(confusion_matrix)
classification_report = metrics.classification_report(y_cut_data,pred_y)
print('\n<============================>\n\n分類報告')
print(classification_report)

print('<============================>\n\n分類準確率')
correct = 0
for i in range(len(y_cut_data)):
    if y_cut_data[i] == pred_y[i]:
        correct += 1
print('測試資料數：', len(y_cut_data), ', 預測正確數：', correct, '準確率：', (correct/len(y_cut_data)))

混淆矩陣
true/predict
[[1308   38  195  151   74   65   40   52   34    3  396   96   40]
 [ 116  906  276  242  325  204   55  183    5   13   91   62   91]
 [  83  182 1350  277   80   48   58   66    3   16  151  185   18]
 [ 181  199  281  867  264  215  124  158   21   40  150  222  108]
 [  61  184   62  173 1255  203   59  127    1   20   66   76   86]
 [ 107  128   44  220  108 1222   48  137   21   46   37   75  181]
 [ 164   69  136  135  358  107  880  192  118   28   45  121  124]
 [ 175  151  119  212  271  268  134  766   30   46   26   87  209]
 [ 296   91  117  355  162  219  163  282  441   33  142  167  147]
 [ 150  154  144  192  111  126   64  114   68  312  352  362  237]
 [ 378  137  118  228  105   41   65   41   34   36  716  245  157]
 [ 120   81  144   90   46   64   23   78   20  120  218 1046  163]
 [ 335  174  256  207   55  147    6   87    3   32  318  235  632]]

<============================>

分類報告
              precision    recall  f1-score   support

    

/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
